# Pulse Length for LiH is 306.9 ns with Strict Partial Compilation

In [1]:
import sys
sys.path.append('../..')
import config
from fqc import uccsd, util

import numpy as np
from datetime import datetime

data_path = config.DATA_PATH
file_name = datetime.today().strftime('%h%d')

In [2]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

In [3]:
d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 6000
decay =  max_iterations / 2
convergence = {'rate':0.01, 'max_iterations': max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay, 'min_grad': 1e-12, 'update_step': 20}
reg_coeffs = {}

In [4]:
N = 4
connected_qubit_pairs = util.get_nearest_neighbor_coupling_list(2, 2, directed=False)
H0 = np.zeros((d ** N, d ** N))
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)

In [6]:
circuit = uccsd.get_uccsd_circuit('LiH')

In [13]:
slices = uccsd.get_uccsd_slices(circuit, granularity=1)
print(len(slices))
print(len([slice for slice in slices if not slice.parameterized]))

81
41


(ran each of the 41 slices via parallel sbatch job array)

In [34]:
import os
times = []
for filename in os.listdir('LiH/'):
    if filename.endswith('.out'):
        lines = open('LiH/%s' % filename).readlines()
        if len(lines) > 5 and lines[-1].startswith('^^^SHORTEST TIME was '):
            result = lines[-1]
            time = float(result[len('^^^SHORTEST TIME was '):])
            if time > 50:
                print(time, filename)
            else:
                times.append(time)
            
print(times)
print(len(times))
print(sum(times))

[5.35, 4.85, 4.85, 9.55, 3.5, 4.85, 9.55, 3.95, 31.45, 3.5, 3.95, 8.15, 7.7, 10.5, 8.6, 9.55, 7.25, 8.15, 10.5, 7.25, 9.1, 7.25, 4.4, 7.25, 10.05, 7.25, 3.5, 8.6, 7.25, 7.7, 3.95, 7.25, 3.95, 8.6, 11.0, 3.95, 7.25, 5.8, 4.4, 8.15, 7.25]
41
306.9


In [24]:
# each parametrized slice is a z gate, which takes
parametrized_times = [np.round(util.circuitutil.get_max_pulse_time(
    slice.circuit), 4) for slice in slices if slice.parameterized]
# (each parametrized slice is just an RZ gate, hence the same time for each one)
print(sum(parametrized_times))

16.000000000000007


In [ ]:
print(sum(parametrized_times) + sum(times))